## Get 16S feature table for 16S and ITS co-occurence. Only keep taxas that are present in at least 10% of all samples. Split two materials, and compare relative VS quantitative

In [1]:
source("./PhyloseqObjects.r")

Warning message in strsplit(conditionMessage(e), "\n"):
“input string 1 is invalid in this locale”

In [2]:
physeq.whole

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 945 taxa and 27 samples ]
sample_data() Sample Data:       [ 27 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 945 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 945 tips and 943 internal nodes ]

In [3]:
physeq.whole.genus = tax_glom(physeq.whole, "Genus")
physeq.whole.genus

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 231 taxa and 27 samples ]
sample_data() Sample Data:       [ 27 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 231 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 231 tips and 230 internal nodes ]

In [4]:
# build a prevalence table for filtering
prevalencedf = apply(X = otu_table(physeq.whole.genus),
                     MARGIN = 1,
                     FUN = function(x){sum(x > 0)})

prevalencedf = data.frame(Prevalence = prevalencedf,
                          TotalAbundance = taxa_sums(physeq.whole.genus))

In [5]:
# filter out features that are not presnet in 10% of all samples
prevalenceThreshold = 0.10 * nsamples(physeq.whole.genus)
prevalenceThreshold
keepTaxa = rownames(prevalencedf)[(prevalencedf$Prevalence >= prevalenceThreshold)]
length(keepTaxa)
#keepTaxa

[1] 2.7

[1] 23

In [7]:
physeq.whole.genus.prevalent = prune_taxa(keepTaxa, physeq.whole.genus)
physeq.whole.genus.prevalent

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 23 taxa and 27 samples ]
sample_data() Sample Data:       [ 27 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 23 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 23 tips and 22 internal nodes ]

### Relative

In [8]:
physeq.whole.genus.prevalent.rel = transform_sample_counts(physeq.whole.genus.prevalent, function(x) 100 * x/sum(x))
sample_sums(physeq.whole.genus.prevalent.rel)

T1R1 T1R10  T1R4  T1R5  T1R7  T1R9  T2R1 T2R10  T2R4  T2R5  T2R7  T2R9  T3R1 
  100   100   100   100   100   100   100   100   100   100   100   100   100 
T3R10  T3R5  T3R7  T3R9  T4R1 T4R10  T4R7  T4R9  T5R1 T5R10  T5R4  T5R5  T5R7 
  100   100   100   100   100   100   100   100   100   100   100   100   100 
 T5R9 
  100

In [9]:
# reduce the name length: p.rel.mdf = physeq.whole.genus.prevelant.rel.mdf
p.rel.mdf = subset_samples(physeq.whole.genus.prevalent.rel, Material == "MDF")
# filter out features that are not present in all samples
p.rel.mdf = prune_taxa(taxa_sums(p.rel.mdf) > 0, p.rel.mdf)
# filter out fatures that are NA at Genus level
p.rel.mdf = subset_taxa(p.rel.mdf, !Genus == "g__")

In [10]:
p.rel.mdf

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 17 taxa and 12 samples ]
sample_data() Sample Data:       [ 12 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 17 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 17 tips and 16 internal nodes ]

In [13]:
write.csv(otu_table(p.rel.mdf), "16S-rel-mdf.csv")
write.csv(tax_table(p.rel.mdf), "16S-mdf-taxa.csv")

In [14]:
p.rel.gyp = subset_samples(physeq.whole.genus.prevalent.rel, Material == "Gypsum")
p.rel.gyp = prune_taxa(taxa_sums(p.rel.gyp) > 0, p.rel.gyp)
p.rel.gyp = subset_taxa(p.rel.gyp, !Genus == "g__")
p.rel.gyp

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 19 taxa and 15 samples ]
sample_data() Sample Data:       [ 15 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 19 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 19 tips and 18 internal nodes ]

In [15]:
write.csv(otu_table(p.rel.gyp), "16S-rel-gyp.csv")
write.csv(tax_table(p.rel.gyp), "16S-gyp-taxa.csv")

### Quantitative

In [16]:
# using previous objects, and convert relative abundance to quantitative abundance
#sample_data(p.rel.mdf)
count.mdf = as.data.frame(sample_data(p.rel.mdf))$Count
p.quan.mdf = p.rel.mdf
otu_table(p.quan.mdf) = rel_to_quan(p.quan.mdf, count.mdf)

In [17]:
write.csv(otu_table(p.quan.mdf), "16S-quan-mdf.csv")
# taxonomy for quantitative data is the same as relative data

In [18]:
#sample_data(p.rel.gyp)
count.gyp = as.data.frame(sample_data(p.rel.gyp))$Count
p.quan.gyp = p.rel.gyp
otu_table(p.quan.gyp) = rel_to_quan(p.quan.gyp, count.gyp)

In [19]:
write.csv(otu_table(p.quan.gyp), "16S-quan-gyp.csv")
# taxonomy for quantitative data is the same as relative data